In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/RecomSysData/movies_metadata.csv')
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## Building a Simple Recommender: (Recommender rely on some simple metric).
our metric will be ((v/v+m) * R) + ((m/v+m) * c) where:


*   v : number of votes garned.
*   m: minimum number of votes required.
*   R: average rating for the movie.
*   C: average rating for all movies.






In [4]:
m = movies.vote_count.quantile(.8)
m

50.0

We will take into account only movies that are greater than 45 minutes and less than 300 minutes.

In [5]:
q_movies = movies[(movies.runtime >= 45) & (movies.runtime <= 300)]
q_movies = q_movies[q_movies.vote_count >= m]

In [6]:
q_movies.shape

(8963, 24)

In [7]:
C = movies.vote_average.mean()
C

5.618207215134185

In [8]:
def weighted_rating(X,m = m, C = C):
  R = X.vote_average
  V = X.vote_count

  return ((V/(V+m)) * R) + ((m/(V + m)) * C)

In [9]:
q_movies['score'] = q_movies.apply(weighted_rating, axis = 1)

In [10]:
top_250 = q_movies[['id', 'title', 'vote_average', 'vote_count', 'score']].sort_values(by = 'score', ascending = False).head(250)
top_250

,id,title,vote_average,vote_count,score
10309,19404,Dilwale Dulhania Le Jayenge,9.1,661.0,8.855148
314,278,The Shawshank Redemption,8.5,8358.0,8.482863
834,238,The Godfather,8.5,6024.0,8.476278
40251,372058,Your Name.,8.5,1030.0,8.366584
12481,155,The Dark Knight,8.3,12269.0,8.289115
...,...,...,...,...,...
26566,283995,Guardians of the Galaxy Vol. 2,7.6,4858.0,7.579811
3080,5924,Papillon,7.8,445.0,7.579617
5016,614,Wild Strawberries,7.9,306.0,7.579523
15348,10193,Toy Story 3,7.6,4710.0,7.579183
